In [117]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np

In [118]:
'''Position limits for the newly introduced products:

- `VOLCANIC_ROCK`: 400

`VOLCANIC_ROCK_VOUCHER_9500` :

- Position Limit: 200
- Strike Price: 9,500 SeaShells
- Expiration deadline: 7 days (1 round = 1 day) starting from round 1

`VOLCANIC_ROCK_VOUCHER_9750` :

- Position Limit: 200
- Strike Price: 9,750 SeaShells
- Expiration deadline: 7 days (1 round = 1 day) starting from round 1

`VOLCANIC_ROCK_VOUCHER_10000` :

- Position Limit: 200
- Strike Price: 10,000 SeaShells
- Expiration deadline: 7 days (1 round = 1 day) starting from round 1

`VOLCANIC_ROCK_VOUCHER_10250` :

- Position Limit: 200
- Strike Price: 10,250 SeaShells
- Expiration deadline: 7 days (1 round = 1 day) starting from round 1

`VOLCANIC_ROCK_VOUCHER_10500` :

- Position Limit: 200
- Strike Price: 10,500 SeaShells
- Expiration deadline: 7 days (1 round = 1 day) starting from round 1'''


class Product:
    ROCK = 'VOLCANIC_ROCK'
    VOUCHER_9500 = 'VOLCANIC_ROCK_VOUCHER_9500'
    VOUCHER_9750 = 'VOLCANIC_ROCK_VOUCHER_9750'
    VOUCHER_10000 = 'VOLCANIC_ROCK_VOUCHER_10000'
    VOUCHER_10250 = 'VOLCANIC_ROCK_VOUCHER_10250'
    VOUCHER_10500 = 'VOLCANIC_ROCK_VOUCHER_10500'

position_limits = {
    Product.ROCK: 400,
    Product.VOUCHER_9500: 200,
    Product.VOUCHER_9750: 200,
    Product.VOUCHER_10000: 200,
    Product.VOUCHER_10250: 200,
    Product.VOUCHER_10500: 200
}

In [119]:
prices = pd.concat([pd.read_csv(f'round3/prices_round_3_day_{i}.csv', sep=';') for i in range(3)], ignore_index=True)
trades = pd.concat([pd.read_csv(f'round3/trades_round_3_day_{i}_nn.csv', sep=';') for i in range(3)], ignore_index=True)

In [120]:
prices['swmid'] = (prices['bid_price_1'] * prices['ask_volume_1'] + prices['ask_price_1'] * prices['bid_volume_1']) / (prices['ask_volume_1'] + prices['bid_volume_1'])

In [121]:
rock = prices[prices["product"] == Product.ROCK].reset_index(drop=True).copy()
voucher_9500 = prices[prices["product"] == Product.VOUCHER_9500].reset_index(drop=True).copy()
voucher_9750 = prices[prices["product"] == Product.VOUCHER_9750].reset_index(drop=True).copy()
voucher_10000 = prices[prices["product"] == Product.VOUCHER_10000].reset_index(drop=True).copy()
voucher_10250 = prices[prices["product"] == Product.VOUCHER_10250].reset_index(drop=True).copy()
voucher_10500 = prices[prices["product"] == Product.VOUCHER_10500].reset_index(drop=True).copy()

In [122]:
[rock, voucher_9500, voucher_9750, voucher_10000, voucher_10250, voucher_10500] = [df.rename(columns={'ask_price_1': 'ask_price', 'bid_price_1': 'bid_price', 'ask_volume_1': 'ask_volume', 'bid_volume_1': 'bid_volume'}) for df in [rock, voucher_9500, voucher_9750, voucher_10000, voucher_10250, voucher_10500]]

In [123]:
rock = rock.drop(columns=['product'], axis=1).rename(columns={col: col + '_rock' for col in rock.columns if col not in ['timestamp', 'day']})
voucher_9500 = voucher_9500.drop(columns=['product'], axis=1).rename(columns={col: col + '_voucher9500' for col in voucher_9500.columns if col not in ['timestamp', 'day']})
voucher_9750 = voucher_9750.drop(columns=['product'], axis=1).rename(columns={col: col + '_voucher9750' for col in voucher_9750.columns if col not in ['timestamp', 'day']})
voucher_10000 = voucher_10000.drop(columns=['product'], axis=1).rename(columns={col: col + '_voucher10000' for col in voucher_10000.columns if col not in ['timestamp', 'day']})
voucher_10250 = voucher_10250.drop(columns=['product'], axis=1).rename(columns={col: col + '_voucher10250' for col in voucher_10250.columns if col not in ['timestamp', 'day']})
voucher_10500 = voucher_10500.drop(columns=['product'], axis=1).rename(columns={col: col + '_voucher10500' for col in voucher_10500.columns if col not in ['timestamp', 'day']})

In [124]:
# join croissants, james, djembes, basket1, basket2 on timestamp
mk = rock.merge(voucher_9500, on=['day', 'timestamp'])
mk = mk.merge(voucher_9750, on=['day', 'timestamp'])
mk = mk.merge(voucher_10000, on=['day', 'timestamp'])
mk = mk.merge(voucher_10250, on=['day', 'timestamp'])
mk = mk.merge(voucher_10500, on=['day', 'timestamp'])
mk

,day,timestamp,bid_price_rock,bid_volume_rock,bid_price_2_rock,bid_volume_2_rock,bid_price_3_rock,bid_volume_3_rock,ask_price_rock,ask_volume_rock,...,bid_volume_3_voucher10500,ask_price_voucher10500,ask_volume_voucher10500,ask_price_2_voucher10500,ask_volume_2_voucher10500,ask_price_3_voucher10500,ask_volume_3_voucher10500,mid_price_voucher10500,profit_and_loss_voucher10500,swmid_voucher10500
0,0,0,10502.0,205.0,NaN,NaN,NaN,NaN,10504,146,...,NaN,100,19,NaN,NaN,NaN,NaN,99.5,0.0,99.5
1,0,100,10509.0,192.0,NaN,NaN,NaN,NaN,10511,122,...,NaN,103,13,NaN,NaN,NaN,NaN,102.5,0.0,102.5
2,0,200,10512.0,167.0,NaN,NaN,NaN,NaN,10514,109,...,NaN,105,15,NaN,NaN,NaN,NaN,104.5,0.0,104.5
3,0,300,10517.0,118.0,10516.0,68.0,NaN,NaN,10518,118,...,NaN,107,13,NaN,NaN,NaN,NaN,106.5,0.0,106.5
4,0,400,10509.0,128.0,10508.0,73.0,NaN,NaN,10510,128,...,NaN,103,15,NaN,NaN,NaN,NaN,102.5,0.0,102.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,2,999500,10167.0,110.0,10166.0,51.0,NaN,NaN,10169,156,...,NaN,4,20,NaN,NaN,NaN,NaN,3.5,0.0,3.2
29996,2,999600,10166.0,161.0,NaN,NaN,NaN,NaN,10168,101,...,NaN,4,14,NaN,NaN,NaN,NaN,3.5,0.0,3.5
29997,2,999700,10166.0,143.0,10165.0,51.0,NaN,NaN,10168,194,...,NaN,4,13,NaN,NaN,NaN,NaN,3.5,0.0,3.5
29998,2,999800,10167.0,119.0,10166.0,57.0,NaN,NaN,10168,119,...,NaN,4,20,NaN,NaN,NaN,NaN,3.5,0.0,3.5


In [125]:
q = mk['day'] == 1
df = mk[q]

fig = go.Figure()

fig.add_trace(go.Scatter(x=df['timestamp'], y=df['mid_price_rock'], mode='lines', name='Rock'))
fig.add_trace(go.Scatter(x=df['timestamp'], y=df['mid_price_voucher10250'], mode='lines', name='Voucher 10000', yaxis='y2'))
fig.update_layout(
    title='Rock and Voucher 10000 Prices',
    xaxis_title='Timestamp',
    yaxis_title='Price',
    yaxis2=dict(
        title='Voucher 10000 Price',
        overlaying='y',
        side='right'
    )
)
fig.show()

In [126]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import brentq

def black_scholes_call(spot, strike, time_to_expiry, volatility):
    d1 = (np.log(spot / strike) + (0.5 * volatility ** 2) * time_to_expiry) / (volatility * np.sqrt(time_to_expiry))
    d2 = d1 - volatility * np.sqrt(time_to_expiry)
    call_price = (spot * norm.cdf(d1) - strike * norm.cdf(d2))
    return call_price

def delta(spot, strike, time_to_expiry, volatility):
    d1 = (np.log(spot) - np.log(strike) + (0.5 * volatility ** 2) * time_to_expiry) / (volatility * np.sqrt(time_to_expiry))
    return norm.cdf(d1)

def gamma(spot, strike, time_to_expiry, volatility):
    d1 = (np.log(spot) - np.log(strike) + (0.5 * volatility ** 2) * time_to_expiry) / (volatility * np.sqrt(time_to_expiry))
    return norm.pdf(d1)/(spot * volatility * np.sqrt(time_to_expiry))

def vega(spot, strike, time_to_expiry, volatility):
    d1 = (np.log(spot) - np.log(strike) + (0.5 * volatility ** 2) * time_to_expiry) / (volatility * np.sqrt(time_to_expiry))
    return norm.pdf(d1) * (spot * np.sqrt(time_to_expiry)) / 100

def implied_volatility(call_price, spot, strike, time_to_expiry):
    # Define the equation where the root is the implied volatility
    def equation(volatility):
        estimated_price = black_scholes_call(spot, strike, time_to_expiry, volatility)
        return estimated_price - call_price

    # Using Brent's method to find the root of the equation
    implied_vol = brentq(equation, 1e-10, 3.0, xtol=1e-10)
    return implied_vol

def realized_vol(df_coconut_call, window, step_size):
    df_coconut_call[f'log_return_{step_size}'] = np.log(df_coconut_call['mid_price_rock'].to_numpy()/df_coconut_call['mid_price_rock'].shift(step_size).to_numpy())
    dt = step_size / 250 / 10000 
    df_coconut_call[f'realized_vol_{step_size}'] = df_coconut_call[f'log_return_{step_size}'].rolling(window=window).apply(lambda x: np.mean(x[::step_size]**2) / dt)
    df_coconut_call[f'realized_vol_{step_size}'] = np.sqrt(df_coconut_call[f'realized_vol_{step_size}'].to_numpy())
    return df_coconut_call

def vanna(spot, strike, time_to_expiry, volatility):
    d1 = (np.log(spot) - np.log(strike) + (0.5 * volatility ** 2) * time_to_expiry) / (volatility * np.sqrt(time_to_expiry))
    return 1/100 * np.sqrt(time_to_expiry) * norm.pdf(d1) * (1- d1/(volatility * np.sqrt(time_to_expiry)))


In [127]:
realized_vol(df, 30, 1)

/var/folders/9p/y7yhf54d2yd5cdpt7hc80f2h0000gn/T/ipykernel_76886/3836841469.py:34: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/9p/y7yhf54d2yd5cdpt7hc80f2h0000gn/T/ipykernel_76886/3836841469.py:36: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/9p/y7yhf54d2yd5cdpt7hc80f2h0000gn/T/ipykernel_76886/3836841469.py:37: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

,day,timestamp,bid_price_rock,bid_volume_rock,bid_price_2_rock,bid_volume_2_rock,bid_price_3_rock,bid_volume_3_rock,ask_price_rock,ask_volume_rock,...,ask_volume_voucher10500,ask_price_2_voucher10500,ask_volume_2_voucher10500,ask_price_3_voucher10500,ask_volume_3_voucher10500,mid_price_voucher10500,profit_and_loss_voucher10500,swmid_voucher10500,log_return_1,realized_vol_1
10000,1,0,10515.0,143.0,10514.0,57.0,NaN,NaN,10517,200,...,12,NaN,NaN,NaN,NaN,95.5,0.0,95.5,NaN,NaN
10001,1,100,10514.0,104.0,10513.0,44.0,NaN,NaN,10516,104,...,13,NaN,NaN,NaN,NaN,94.5,0.0,94.5,-0.000095,NaN
10002,1,200,10514.0,133.0,10513.0,62.0,NaN,NaN,10516,133,...,16,NaN,NaN,NaN,NaN,94.5,0.0,94.5,0.000000,NaN
10003,1,300,10516.0,200.0,NaN,NaN,NaN,NaN,10518,149,...,14,NaN,NaN,NaN,NaN,95.5,0.0,95.5,0.000190,NaN
10004,1,400,10519.0,108.0,10518.0,63.0,NaN,NaN,10520,103,...,18,NaN,NaN,NaN,NaN,97.5,0.0,97.5,0.000238,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,1,999500,10224.0,162.0,NaN,NaN,NaN,NaN,10225,8,...,18,NaN,NaN,NaN,NaN,9.5,0.0,9.5,0.000245,0.371418
19996,1,999600,10225.0,130.0,10224.0,67.0,NaN,NaN,10226,130,...,19,NaN,NaN,NaN,NaN,9.5,0.0,9.5,0.000098,0.372490
19997,1,999700,10221.0,162.0,NaN,NaN,NaN,NaN,10223,108,...,16,NaN,NaN,NaN,NaN,9.5,0.0,9.5,-0.000342,0.378863
19998,1,999800,10223.0,122.0,10222.0,65.0,NaN,NaN,10224,116,...,16,NaN,NaN,NaN,NaN,9.5,0.0,9.5,0.000147,0.378866


In [130]:
df['realized_vol_1'].mean()

np.float64(0.3801745659351543)

In [ ]:
trading_dte = 8 - mk['day']
mk['tte'] = 1 - mk['timestamp'] / (1_000_000 * trading_dte)

In [ ]:
spot_price = 10000        # Spot price of the underlying asset
strike_price = 10000      # Strike price of the option
call_price = 637.5         # Market price of the call option
time_to_expiry = 1      # Time to expiry in years
initial_guess = 16

trading_dte = 8 - mk['day']
mk['time_to_expiry'] = 1 - mk['timestamp'] / (1_000_000 * trading_dte)
mk['implied_vol'] = mk.apply(lambda row: implied_volatility(row['mid_price_voucher10000'], row['mid_price_rock'], strike_price, row['time_to_expiry']), axis=1)

df_coconut_call['implied_vol'] = df_coconut_call.apply(lambda row: implied_volatility(row['mid_price'], row['mid_price_coconut'], strike_price, row['time_to_expiry']), axis=1)
df_coconut_call['delta'] = df_coconut_call.apply(lambda row: delta(row['mid_price_coconut'], strike_price, row['time_to_expiry'], row['implied_vol']), axis=1)
df_coconut_call['gamma'] = df_coconut_call.apply(lambda row: gamma(row['mid_price_coconut'], strike_price, row['time_to_expiry'], row['implied_vol']), axis=1)
df_coconut_call['vega'] = df_coconut_call.apply(lambda row: vega(row['mid_price_coconut'], strike_price, row['time_to_expiry'], row['implied_vol']), axis=1)

In [ ]:
spot_price = 10000        # Spot price of the underlying asset
strike_price = 10000      # Strike price of the option
call_price = 637.5         # Market price of the call option
time_to_expiry = 1      # Time to expiry in years
initial_guess = 16
df_coconut_call['implied_vol'] = df_coconut_call.apply(lambda row: implied_volatility(row['mid_price'], row['mid_price_coconut'], strike_price, time_to_expiry), axis=1)
df_coconut_call['delta'] = df_coconut_call.apply(lambda row: delta(row['mid_price_coconut'], strike_price, time_to_expiry, row['implied_vol']), axis=1)
df_coconut_call['gamma'] = df_coconut_call.apply(lambda row: gamma(row['mid_price_coconut'], strike_price, time_to_expiry, row['implied_vol']), axis=1)
df_coconut_call['vega'] = df_coconut_call.apply(lambda row: vega(row['mid_price_coconut'], strike_price, time_to_expiry, row['implied_vol']), axis=1)